# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
#create an api request to citybik.es to get the information from Hamilton Ontario
import requests
import json
import pandas as pd

#make the request
response = requests.get('https://api.citybik.es/v2/networks')

#convert the response to a json object
data = response.json()

#convert the json object to a pandas dataframe
df = pd.DataFrame(data['networks'])



In [2]:
df.head()

,company,href,id,location,name,source,gbfs_href,license,ebikes
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,"{'city': 'Moscow', 'country': 'RU', 'latitude'...",Velobike,NaN,NaN,NaN,NaN
1,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,"{'city': 'Siena', 'country': 'IT', 'latitude':...",Bicincittà,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN
2,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,"{'city': 'Maroussi', 'country': 'GR', 'latitud...",Cyclopolis,NaN,NaN,NaN,NaN
3,[Cyclopolis Systems],/v2/networks/cyclopolis-nafplio,cyclopolis-nafplio,"{'city': 'Nafplio', 'country': 'GR', 'latitude...",Cyclopolis,NaN,NaN,NaN,NaN
4,[Comunicare S.r.l.],/v2/networks/bicincitta-parco-dei-colli-di-ber...,bicincitta-parco-dei-colli-di-bergamo,"{'city': 'Parco dei Colli di Bergamo', 'countr...",Bicincittà,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN


In [3]:
#json normalization of the location column
location = pd.json_normalize(df['location'])
location.columns = ['location.' + str(col) for col in location.columns]
df = df.drop('location', axis=1)
df = pd.concat([df, location], axis=1)


In [4]:
df.head()

,company,href,id,name,source,gbfs_href,license,ebikes,location.city,location.country,location.latitude,location.longitude
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,NaN,NaN,NaN,NaN,Moscow,RU,55.750000,37.616667
1,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,Bicincittà,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,Siena,IT,43.318600,11.330600
2,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,Cyclopolis,NaN,NaN,NaN,NaN,Maroussi,GR,38.056872,23.808330
3,[Cyclopolis Systems],/v2/networks/cyclopolis-nafplio,cyclopolis-nafplio,Cyclopolis,NaN,NaN,NaN,NaN,Nafplio,GR,37.563940,22.809340
4,[Comunicare S.r.l.],/v2/networks/bicincitta-parco-dei-colli-di-ber...,bicincitta-parco-dei-colli-di-bergamo,Bicincittà,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,Parco dei Colli di Bergamo,IT,45.722956,9.649230


In [6]:
#filter the dataframe to only include id 'sobi-hamilton'
hamilton = df[df['id'] == 'sobi-hamilton']
hamilton = hamilton.reset_index(drop=True)
hamilton.head()

,company,href,id,name,source,gbfs_href,license,ebikes,location.city,location.country,location.latitude,location.longitude
0,[Social Bicycles Inc.],/v2/networks/sobi-hamilton,sobi-hamilton,SoBi,NaN,https://hamilton.socialbicycles.com/opendata/g...,NaN,NaN,"Hamilton, ON",CA,43.256436,-79.869297


In [7]:
#create a new request to get the information for the Hamilton Social Bicycles inc. api using the link from the gbfs_href column
response = requests.get(hamilton['gbfs_href'][0])
data = response.json()
sobi_df = pd.DataFrame(data['data']['en']['feeds'])
sobi_df.head()

,name,url
0,gbfs,http://hamilton.socialbicycles.com/opendata/gb...
1,system_information,http://hamilton.socialbicycles.com/opendata/sy...
2,station_information,http://hamilton.socialbicycles.com/opendata/st...
3,station_status,http://hamilton.socialbicycles.com/opendata/st...
4,free_bike_status,http://hamilton.socialbicycles.com/opendata/fr...


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

Put your parsed results into a DataFrame.